In [ ]:
# pip install llama-index
# !pip install llama-index-embeddings-huggingface
# !pip install llama-index-llms-huggingface
# !pip install llama-index-postprocessor-cohere-rerank

In [ ]:
import os
os.environ['HF_HOME'] = '' # please specify your path.

In [ ]:
import os
from llama_index.postprocessor.cohere_rerank import CohereRerank

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

# documents = SimpleDirectoryReader("data").load_data()
# index = VectorStoreIndex.from_documents(documents)
documents = SimpleDirectoryReader(
    "./dictionary_genemore/" # please specific your pathway
).load_data()
index = VectorStoreIndex.from_documents(documents)

In [ ]:
from huggingface_hub import login

login("") # please use your own code

In [ ]:
from llama_index.core import ServiceContext
import transformers

from llama_index.core.llms import LLM
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import Settings

In [ ]:
# setup prompts - specific to StableLM
from llama_index.core import PromptTemplate

# This will wrap the default prompts that are internal to llama-index
# taken from https://huggingface.co/Writer/camel-5b-hf
query_wrapper_prompt = PromptTemplate(
    "Below is an instruction that describes a task. "
    "Write a response that appropriately completes the request. Please do not share information if you do not know the answer.\n\n"
    "### Instruction:\n{query_str}\n\n### Response:"
)

In [ ]:
# !pip install transformers --upgrade

In [ ]:
llm = HuggingFaceLLM(
    context_window=2048,
    max_new_tokens=1024,
    generate_kwargs={"temperature": 0.25, "do_sample": False},
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="google/gemma-7b-it",
    model_name="google/gemma-7b-it",
    device_map="auto",
    tokenizer_kwargs={"max_length": 2048},
    
    # uncomment this if using CUDA to reduce memory usage
    # model_kwargs={"torch_dtype": torch.float16}
)

Settings.chunk_size = 512
Settings.llm = llm

In [ ]:
index = VectorStoreIndex.from_documents(documents)

In [ ]:
# # set Logging to DEBUG for more detailed outputs
# query_engine = index.as_query_engine()
# response = query_engine.query(f"Please summarize the major function of gene: ZNF385A. Use academic language in one paragraph and include pathway information.")
# print(response)

In [ ]:
# # set Logging to DEBUG for more detailed outputs
# query_engine = index.as_query_engine()
# response = query_engine.query(f"Please summarize the major function of gene: ZNF385A. Use academic language in one paragraph and include pathway information.")
# print(response)

In [ ]:
gene_list = [
    "HEATR5B",
"ZNF385A",
"ZNF226",
"IGKV2D-36",
"RPL37P24",
"FGF7",
"LRRC7",
"MIR6721",
"SRD5A3",
"MIR608",
"EOGT",
"CASP10",
"CYCSP32",
"HSPA1L",
"SLC30A10",
"GLI1",
"RPS4XP1",
"RPS27AP8",
"CCL23",
"ACSS3"
]
top_k = 1
query_engine = index.as_query_engine(similarity_top_k=top_k)

In [ ]:
for item in gene_list:
    # Test out a query in natural
    print(item)
    response = query_engine.query(f"Please summarize the major function of gene: {item}. Use academic language in one paragraph and include gene name and pathway information.")
    print(response.response)